In [1]:
import pygrib 
import numpy as np 

GRIB_PATH = r'GPOSNMC20170202122017020312P.grib2'
grib = pygrib.open(GRIB_PATH)

for msg in grib:
    print(msg)

1:Orography:m (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
2:Land-sea mask:(0 - 1) (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
3:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
4:U component of wind:m s**-1 (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
5:V component of wind:m s**-1 (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
6:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 24 hrs:from 201702021200
7:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 92500 Pa:fcst time 24 hrs:from 201702021200
8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 24 hrs:from 201702021200
9:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 77500 Pa:fcst time 24 hrs:from 201702021200
10:U component of wind:m s**-1 (instant):regular_ll:isobari

79:Temperature:K (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201702021200
80:Temperature:K (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 24 hrs:from 201702021200
81:Temperature:K (instant):regular_ll:isobaricInhPa:level 92500 Pa:fcst time 24 hrs:from 201702021200
82:Temperature:K (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 24 hrs:from 201702021200
83:Temperature:K (instant):regular_ll:isobaricInhPa:level 77500 Pa:fcst time 24 hrs:from 201702021200
84:Temperature:K (instant):regular_ll:isobaricInhPa:level 70000 Pa:fcst time 24 hrs:from 201702021200
85:Temperature:K (instant):regular_ll:isobaricInhPa:level 50000 Pa:fcst time 24 hrs:from 201702021200
86:Temperature:K (instant):regular_ll:isobaricInhPa:level 40000 Pa:fcst time 24 hrs:from 201702021200
87:Temperature:K (instant):regular_ll:isobaricInhPa:level 30000 Pa:fcst time 24 hrs:from 201702021200
88:Temperature:K (instant):regular_ll:isobaricInhPa:level 25000 Pa:fcst time 24 hrs:from 20170

In [2]:
# Obter o nome das variáveis
def get_names(grib):
    grib.seek(0)
    names = []
    for g in grib:
        names.append(g['name'])
    return names
    
# Retornar o nome das variáveis contadas
def count_vars(grib):
    grib.seek(0)
    var_counted = []
    # Pegar o nome das variáveis
    names = get_names(grib)
    for n in names:
        if not var_counted or var_counted[-1][0] != n:
            var_counted.append((n, names.count(n)))
    
    return var_counted


def get_names_units(grib):
    grib.seek(0)
    names_units =[]
    
    for num, g in enumerate(grib):
        if not names_units or names_units[-1][0] != g['name']:
            names_units.append((g["name"], g["units"]))
            
    return names_units


counted_vars = count_vars(grib)
names_units = get_names_units(grib)

print(counted_vars)

[(u'Orography', 1), (u'Land-sea mask', 1), (u'Surface pressure', 1), (u'U component of wind', 19), (u'V component of wind', 19), (u'U component of wind', 19), (u'V component of wind', 19), (u'Vertical velocity', 18), (u'Geopotential Height', 18), (u'Pressure reduced to MSL', 1), (u'Temperature', 19), (u'2 metre temperature', 1), (u'Maximum temperature', 1), (u'Minimum temperature', 1), (u'Relative humidity', 19), (u'Specific humidity', 1), (u'Relative humidity', 19), (u'Precipitable water', 1), (u'10 metre U wind component', 1), (u'10 metre V wind component', 1), (u'unknown', 3), (u'Convective precipitation (water)', 1), (u'Latent heat net flux', 1), (u'Sensible heat net flux', 1), (u'Downward long-wave radiation flux', 1), (u'Upward long-wave radiation flux', 1), (u'Downward short-wave radiation flux', 1), (u'Upward short-wave radiation flux', 1), (u'Net short-wave radiation flux (top of atmosphere)', 1), (u'Net long wave radiation flux (top of atmosphere)', 1), (u'Soil Temperature', 

In [3]:
header = []
header.append([GRIB_PATH])
msg = grib[1]

# Checar se a mensagem é classificada em nível de pressão
def has_levels(type_level):
    if type_level != 'isobaricInhPa': 
        return False
    else: 
        return True

# Obter os levels de pressão
def get_levels(grib):
    grib.seek(0)
    for msg in grib:
        if has_levels(msg['typeOfLevel']):
            selected = grib.select(name=msg['name'], typeOfLevel=msg['typeOfLevel'])
            levels = [grb['level'] for grb in selected]
            return levels

def get_start_lat(msg):
    lat = msg['latitudeOfFirstGridPointInDegrees']
    return lat

def get_start_lon(msg):
    lon = msg['longitudeOfFirstGridPointInDegrees']
    return lon

def get_i_increment(msg):
    return msg['iDirectionIncrementInDegrees']

def get_j_increment(msg):
    return msg['jDirectionIncrementInDegrees']

def count_lon_lat(msg):
    num_lats = len(msg['latitudes'])
    num_lons = len(msg['longitudes'])
    return (num_lons, num_lats)

def get_datadate(msg):
    return msg['dataDate']

def get_specific_height(msg):
    if has_levels(msg['typeOfLevel']):
        return 0
    else:
        return msg['level']


xdef = [count_lon_lat(msg)[0], get_start_lon(msg), get_i_increment(msg)]
ydef = [count_lon_lat(msg)[1], get_start_lat(msg), get_j_increment(msg)]
tdef = [get_datadate(msg)]
# zdef = [len(get_levels(grib), get_levels(grib))]

header.append(xdef)
header.append(ydef)
header.append(tdef)

# Erro com os Levels e Nome das variáveis 

In [4]:
# header
levels = get_levels(grib)
print(levels)
print(len(levels))

[1000, 925, 850, 775, 700, 500, 400, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10, 3]
18


In [5]:
names = get_names(grib)
grib.seek(0)
vars_grib = []

# print(names)
# Adicionar as variáveis em uma lista
for msg in grib:
    if not vars_grib or vars_grib[-1][5] != msg['name']:
        vars_grib.append([msg['shortName'], names.count(msg['name']),
                         msg['indicatorOfParameter'], 'LTYPE', get_specific_height(msg),
                         msg['name'], '(%s)' % msg['units'] ])
        
for v in vars_grib:
    print(v)

RuntimeError: Key/value not found

In [ ]:
msg

In [ ]:
for msg in grib[1:3]:
    print(msg)
    try:
        for k in msg.keys():
            print("%s -> %s" % (msg[k], k))
    except RuntimeError:
        print('ERROR')
        print('\n')